In [1]:
from glob import glob
from os.path import join, basename

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
#common_path = "/Volumes/MMIS-Saraiv/Datasets/Newcastle/EC/sLORETA/LORETA_0,5Hz-resolution"
#common_path = "/Volumes/MMIS-Saraiv/Datasets/Istambul/LORETA/LORETA_0,5Hz-resolution"
#common_path = "/Volumes/MMIS-Saraiv/Datasets/Miltiadous Dataset/LORETA/LORETA_0,5Hz-resolution"
common_path = "/Volumes/MMIS-Saraiv/Datasets/BrainLat/LORETA/LORETA_0,5Hz-resolution/CL"
#out_common_path = "/Volumes/MMIS-Saraiv/Datasets/Newcastle/EC/features_source_ind-bands"
#out_common_path = "/Volumes/MMIS-Saraiv/Datasets/Istambul/features_source_ind-bands"
#out_common_path = "/Volumes/MMIS-Saraiv/Datasets/Miltiadous Dataset/features_source_ind-bands"
out_common_path = "/Volumes/MMIS-Saraiv/Datasets/BrainLat/features_source_ind-bands/CL"
header = ['Frontal', 'Central', 'Parietal', 'Occipital', 'Temporal', 'Limbic']
freqs = np.arange(0.5, 45.5, 0.5)  # these are the lower bound frequencies of 0.5Hz bins

In [3]:
# Read IAF and TF values
#iaf_tf_path = "/Volumes/MMIS-Saraiv/Datasets/Istambul/iaf_tf_my-boudaries.csv"
#iaf_tf_path = "/Volumes/MMIS-Saraiv/Datasets/Miltiadous Dataset/iaf_tf_my-boudaries_AR.csv"
iaf_tf_path = "/Volumes/MMIS-Saraiv/Datasets/BrainLat/iaf_tf_my-boudaries_CL.csv"
#iaf_tf_path = "/Volumes/MMIS-Saraiv/Datasets/Newcastle/EC/iaf_tf_my-boudaries.csv"
#iaf_tf_path = "/Volumes/MMIS-Saraiv/Datasets/Izmir/iaf_tf_my-boudaries.csv"
all_iaf_tf = pd.read_csv(iaf_tf_path, index_col=0)

In [4]:
all_subjects = glob(join(common_path, '*Roi.txt'))

In [5]:
def plot_individual_bands(bands: dict[str, tuple[float, float]], tf: float, iaf: float, subject_code: str):
    # Make a horizonal plot with the bands on the x-axis
    fig, ax = plt.subplots(1, 1, figsize=(20, 3))
    for i, (band, (inf, sup)) in enumerate(bands.items()):
        # coarse line
        plt.hlines(i, inf, sup, label=band, color='black', lw=4)
    # plot vertically dashed in red the TF and IAF
    plt.vlines(tf, -1, len(bands), color='red', linestyles='dashed', lw=2, label='TF')
    plt.vlines(iaf, -1, len(bands), color='blue', linestyles='dashed', lw=2, label='IAF')
    ax.set_yticks(range(len(bands)))
    ax.set_yticklabels(list(bands.keys()))
    ax.set_xlabel("Frequency (Hz)")
    # make all x-ticks from 'bands' and add TF and IAF
    xticks = list(bands.values())
    xticks.append((tf, tf))
    xticks.append((iaf, iaf))
    ax.set_xticks([x for x, _ in xticks])
    ax.set_title("Individual bands of subject " + subject_code)
    #plt.show()
    plt.savefig(join(out_common_path, f"{subject_code}_bands.png"), dpi=300, bbox_inches='tight')
    plt.close()

In [6]:
def average_band(_df, inf, sup, label, verbose=False):
    if verbose:
        print("Inf:", inf, "Sup:", sup)
        
    # Discretize input
    # If inf or sup are not .0 or .5, round to the nearest .0 or .5
    inf = round(inf*2)/2
    sup = round(sup*2)/2
    
    # Average by column
    res = _df.loc[inf: sup-0.5].mean(axis=0)
    
    # Print the index of all rows that were averaged
    bins = _df.loc[inf: sup-0.5].index.tolist()
    n_bins = len(bins)
    if verbose:
        print(label, "| Averaging", bins)
        
    return res, pd.Series({l: n_bins for l in header})

In [7]:
def flatten_df(_df):
    # E.g. Delta_Frontal, Delta_Central, Delta_Parietal, Delta_Occipital, Delta_Temporal, Delta_Limbic, ...
    _res = {}
    for band, band_df in _df.items():
        for region, value in band_df.items():
            _res[f"{band}_{region}"] = value
    _res = pd.DataFrame(_res, index=[0])
    return _res

In [8]:
res = pd.DataFrame(dtype=float)
res_number_bins = pd.DataFrame(dtype=int)
skipped_subjects = 0
low_tf_subjects = 0
for filepath in all_subjects:
    #subject_code = basename(filepath).split('-')[0].split(' ')[1]
    #subject_code = basename(filepath).split('-')[0]
    #subject_code = basename(filepath).split('-')[0].split('PARTICIPANT')[1]
    subject_code = '-'.join(basename(filepath).split('-')[0:2])
    print("Subject code:", subject_code)
    subject_out_filepath = join(out_common_path, f"{subject_code}.csv")
    
    # Read .txt file to Dataframe
    df = pd.read_csv(filepath, sep='  ', header=None, engine='python')
    df.columns = header
    df.index = freqs
    assert df.shape == (90, 6)
    
    # Get IAF and TF of this subject
    #iaf, tf = all_iaf_tf.loc[int(subject_code)]
    iaf, tf = all_iaf_tf.loc[subject_code]
    print("IAF:", iaf, "TF:", tf)
    iaf, tf = round(iaf, 1), round(tf, 1)  # round to 1 decimal place
    
    # Extract/average values per individual band
    band_features = {}
    band_bins = {}
    
    if tf >= iaf:
        print("TF is greater than or equal to IAF. Skipping...")
        skipped_subjects += 1
        continue
     
    if tf < 4.5:
        print("TF is less than 4Hz, hence no Delta band")#, Skipping...")
        low_tf_subjects += 1
        #continue
    
    # Delta
    delta_inf = tf - 4 if tf - 4 >= 2 else 2
    delta_sup = tf-2 if tf-2 > delta_inf else 3
    band_features['Delta'], band_bins['Delta'] = average_band(df, delta_inf, delta_sup, 'Delta', verbose=True)
    # Alpha 1
    alpha1_sup = tf + ((iaf - tf) / 2)
    alpha1_inf = tf if alpha1_sup - tf >= 0.5 else alpha1_sup - 0.5
    if alpha1_inf < 6:
        alpha1_inf = 6 
        alpha1_sup = 6.5
    band_features['Alpha1'], band_bins['Alpha1'] = average_band(df, alpha1_inf, alpha1_sup, 'Alpha1', verbose=True)
    # Theta
    theta_inf = tf-2 if tf-2 >= delta_sup else delta_sup
    theta_sup = tf if tf - theta_inf >= 2 else alpha1_inf
    if theta_sup > alpha1_inf:
        theta_sup = alpha1_inf
    band_features['Theta'], band_bins['Theta'] = average_band(df, theta_inf, theta_sup, 'Theta', verbose=True)
    # Alpha 2
    alpha2_inf = alpha1_sup
    alpha2_sup = iaf if iaf - alpha2_inf >= 0.5 else alpha2_inf + 1.0
    band_features['Alpha2'], band_bins['Alpha2'] = average_band(df, alpha2_inf, alpha2_sup, 'Alpha2', verbose=True)
    # Alpha 3
    alpha3_inf = iaf if iaf >= alpha2_sup else alpha2_sup
    alpha3_sup = iaf + 2 if iaf + 2 <= 13 else 13
    band_features['Alpha3'], band_bins['Alpha3'] = average_band(df, alpha3_inf, alpha3_sup, 'Alpha3')
    # Beta 1
    beta1_inf = 14
    beta1_sup = 20
    band_features['Beta1'], band_bins['Beta1'] = average_band(df, beta1_inf, beta1_sup, 'Beta1')
    # Beta 2
    beta2_inf = 20
    beta2_sup = 30
    band_features['Beta2'], band_bins['Beta2'] = average_band(df, beta2_inf, beta2_sup, 'Beta2')
    # Gamma
    gamma_inf = 30
    gamma_sup = 40
    band_features['Gamma'], band_bins['Gamma'] = average_band(df, gamma_inf, gamma_sup, 'Gamma')
    
    # Plot individual bands
    plot_individual_bands({'Delta': (delta_inf, delta_sup), 'Theta': (theta_inf, theta_sup), 'Alpha1': (alpha1_inf, alpha1_sup), 
                           'Alpha2': (alpha2_inf, alpha2_sup), 'Alpha3': (alpha3_inf, alpha3_sup), 'Beta1': (beta1_inf, beta1_sup)},
                          tf, iaf, subject_code)
    
    # Flatten band_features
    subject_res = flatten_df(band_features)
    subject_res.index = [subject_code]
    
    subject_res_bins = flatten_df(band_bins)
    subject_res_bins.index = [subject_code] 
    
    # Add to res using concat
    res = pd.concat([res, subject_res], axis=0)
    res_number_bins = pd.concat([res_number_bins, subject_res_bins], axis=0) 

Subject code: sub-10001
IAF: 10.5 TF: 6.5
Inf: 2.5 Sup: 4.5
Delta | Averaging [2.5, 3.0, 3.5, 4.0]
Inf: 6.5 Sup: 8.5
Alpha1 | Averaging [6.5, 7.0, 7.5, 8.0]
Inf: 4.5 Sup: 6.5
Theta | Averaging [4.5, 5.0, 5.5, 6.0]
Inf: 8.5 Sup: 10.5
Alpha2 | Averaging [8.5, 9.0, 9.5, 10.0]
Subject code: sub-10005
IAF: 8.5 TF: 5.0
Inf: 2 Sup: 3.0
Delta | Averaging [2.0, 2.5]
Inf: 6 Sup: 6.5
Alpha1 | Averaging [6.0]
Inf: 3.0 Sup: 5.0
Theta | Averaging [3.0, 3.5, 4.0, 4.5]
Inf: 6.5 Sup: 8.5
Alpha2 | Averaging [6.5, 7.0, 7.5, 8.0]
Subject code: sub-10008
IAF: 8.5 TF: 5.0
Inf: 2 Sup: 3.0
Delta | Averaging [2.0, 2.5]
Inf: 6 Sup: 6.5
Alpha1 | Averaging [6.0]
Inf: 3.0 Sup: 5.0
Theta | Averaging [3.0, 3.5, 4.0, 4.5]
Inf: 6.5 Sup: 8.5
Alpha2 | Averaging [6.5, 7.0, 7.5, 8.0]
Subject code: sub-30003
IAF: 9.0 TF: 6.5
Inf: 2.5 Sup: 4.5
Delta | Averaging [2.5, 3.0, 3.5, 4.0]
Inf: 6.5 Sup: 7.75
Alpha1 | Averaging [6.5, 7.0, 7.5]
Inf: 4.5 Sup: 6.5
Theta | Averaging [4.5, 5.0, 5.5, 6.0]
Inf: 7.75 Sup: 9.0
Alpha2 | Avera

In [9]:
print("Skipped subjects:", skipped_subjects)
print("Low TF subjects:", low_tf_subjects)

Skipped subjects: 0
Low TF subjects: 2


In [10]:
res

,Delta_Frontal,Delta_Central,Delta_Parietal,Delta_Occipital,Delta_Temporal,Delta_Limbic,Alpha1_Frontal,Alpha1_Central,Alpha1_Parietal,Alpha1_Occipital,...,Beta2_Parietal,Beta2_Occipital,Beta2_Temporal,Beta2_Limbic,Gamma_Frontal,Gamma_Central,Gamma_Parietal,Gamma_Occipital,Gamma_Temporal,Gamma_Limbic
sub-10001,4.604238,3.432894,2.740924,1.689838,2.398315,2.038025,1.718623,1.220738,1.060271,0.722770,...,0.254146,0.237574,0.452388,0.247907,0.361077,0.216392,0.158957,0.190366,0.358619,0.132024
sub-10005,3.868301,2.393669,1.741347,1.736360,3.085749,1.342220,1.260224,1.012642,1.273962,1.375097,...,0.159734,0.206207,0.307519,0.161002,0.167466,0.097128,0.087515,0.157003,0.209524,0.067436
sub-10008,7.305127,3.417782,1.349074,1.110988,3.645446,1.945547,2.229354,1.099325,0.528266,0.459040,...,0.138285,0.118314,0.489924,0.493099,1.317721,0.234275,0.093076,0.099295,0.380292,0.311362
sub-30003,1.040717,0.775787,0.653944,0.655159,0.899087,0.452050,0.619771,0.374177,0.302586,0.321526,...,0.273029,0.250818,1.656641,0.437361,2.727407,0.805429,0.328962,0.318427,2.364263,0.575654
sub-30005,4.463923,2.913378,1.543316,1.142529,2.043414,2.166201,2.740876,1.730918,0.977636,0.629258,...,0.131665,0.131518,0.180559,0.165792,0.199779,0.107872,0.070776,0.112322,0.164369,0.076619
sub-30006,2.842169,1.505278,1.426757,1.579404,2.956682,1.663766,1.393590,0.770192,0.688666,0.741552,...,0.213948,0.230624,1.158710,0.322068,1.571547,0.513964,0.230226,0.244257,1.442287,0.364866
sub-30007,5.442134,4.375361,2.542282,1.913941,3.055221,2.077249,1.941151,1.750650,0.977131,0.822088,...,0.308991,0.730755,0.415768,0.301067,0.360526,0.170149,0.220437,0.630857,0.364860,0.192695
sub-30010,3.537976,1.532603,0.997981,0.861286,1.260115,1.286618,1.564489,0.799423,0.565340,0.534091,...,0.145961,0.131941,0.280175,0.150884,0.327629,0.154744,0.099318,0.112525,0.277697,0.099962
sub-30014,8.664307,3.674966,2.008838,1.502604,4.242830,2.628864,2.085431,1.301864,0.891584,0.829979,...,0.152680,0.107276,0.329595,0.202966,0.505849,0.184467,0.078281,0.056157,0.290952,0.133938
sub-30016,5.333565,2.283668,1.387504,1.433709,3.104878,1.690350,2.140522,0.983075,0.624049,0.664712,...,0.242904,0.341287,1.053750,0.421221,1.968739,0.462145,0.241218,0.360211,1.233329,0.498193


In [11]:
res_number_bins

,Delta_Frontal,Delta_Central,Delta_Parietal,Delta_Occipital,Delta_Temporal,Delta_Limbic,Alpha1_Frontal,Alpha1_Central,Alpha1_Parietal,Alpha1_Occipital,...,Beta2_Parietal,Beta2_Occipital,Beta2_Temporal,Beta2_Limbic,Gamma_Frontal,Gamma_Central,Gamma_Parietal,Gamma_Occipital,Gamma_Temporal,Gamma_Limbic
sub-10001,4,4,4,4,4,4,4,4,4,4,...,20,20,20,20,20,20,20,20,20,20
sub-10005,2,2,2,2,2,2,1,1,1,1,...,20,20,20,20,20,20,20,20,20,20
sub-10008,2,2,2,2,2,2,1,1,1,1,...,20,20,20,20,20,20,20,20,20,20
sub-30003,4,4,4,4,4,4,3,3,3,3,...,20,20,20,20,20,20,20,20,20,20
sub-30005,2,2,2,2,2,2,1,1,1,1,...,20,20,20,20,20,20,20,20,20,20
sub-30006,4,4,4,4,4,4,2,2,2,2,...,20,20,20,20,20,20,20,20,20,20
sub-30007,1,1,1,1,1,1,1,1,1,1,...,20,20,20,20,20,20,20,20,20,20
sub-30010,4,4,4,4,4,4,2,2,2,2,...,20,20,20,20,20,20,20,20,20,20
sub-30014,1,1,1,1,1,1,1,1,1,1,...,20,20,20,20,20,20,20,20,20,20
sub-30016,3,3,3,3,3,3,1,1,1,1,...,20,20,20,20,20,20,20,20,20,20


In [12]:
# Save res
res.to_csv(join(out_common_path, "all_features.csv"))